# resource

db resources
- pypl-bods.rsh_row_gds_focus
- pypl-edw.pp_scratch
- pypl-edw.pp_scratch_risk
- pypl-edw.pp_oap_guxia_t

# code

## date manipulation 

In [ ]:
-- format date as string
select format_date('%Y-%m', some_date_col)

-- extract date part
select date_trunct(some_date_col, month/day/hour)

-- foramt datetime as string
select format_datetime('%Y-%m-%d %H:%M:%S', datetime_col)

-- format string as date
select parse_date('%Y-%m-%d', some_str_date_col)

## timestamp manipulation 

In [ ]:
-- format timestamp as string
select format_timestamp('%Y-%m-%d %H:%M:%S', timestamp_millis(1654555414757))

-- parse timesstamp from string
select parse_timestamp('%Y-%m-%d', '2023-08-23')

-- convert datetime to timestamp
-- last argument is timezone value
select timestamp(some_datetime_col, 'UTF-07') 

-- convert interger to timestamp
select timestamp_millis(1514761200000)

## create table as select, with partition columns

In [ ]:
create table table_name
partition by some_dt_column
as (
    with sub_table,
    sub_table_2
    select * from sub_table2
)
;

## create table if not exists

In [ ]:
drop table if exists pypl-bods.rsh_row_gds_focus.table_name;

create table pypl-bods.rsh_row_gds_focus.table_name (
    user_id string
    , ato_claim_dt date
    , category string
)
;

## list all db in project

In [ ]:
select
    schema_name
from
    pypl-edw.INFORMATION_SCHEMA.SCHEMATA

## search table in DB

In [ ]:
select
    table_name
from pypl-edw.pp_scratch.INFORMATION_SCHEMA.TABLES
where
    table_name like 'dim_customer'

## get table meta data

In [ ]:
# get meta info
from datetime import datetime

table_full_schema="pypl-edw.pp_scratch.dim_customer"
today = datetime.now().strftime('%Y-%m-%d')

db=".".join(table_full_schema.split('.')[:-1]).strip()
table_name = table_full_schema.split('.')[-1].strip()

sql_cols = f"""
select *
from {db}.INFORMATION_SCHEMA.COLUMNS
where
    table_name like '{table_name}'
"""
ret = %ppbq $sql_cols
ret[['column_name', 'data_type', 'is_partitioning_column']].to_csv('schema.csv', index=False)


# sample
date_col = 'pmt_start_date'
sql_sample = f"""
select *
from 
    {table_full_schema}
where
    1=1
    and {date_col} between date_sub('{today}', interval 15 day) and '{today}'
limit 100
"""

ret = %ppbq $sql_sample
ret.T.to_csv('samples.csv')

## string split

In [ ]:
select cast(split(brm_bad_tag_assigned, '_')[ordinal(1)] as string)

## percentile

In [ ]:
select approx_quantiles(some_value_column, 100)[offset(99)] as p99

## column equality stats

In [ ]:
select
    '{category}' as category
    , '{a}' as asset_a
    , '{b}' as asset_b
    
    , case when {a} is null or length(trim({a})) = 0 then 1 else 0 end as asset_a_is_null
    , case when {b} is null or length(trim({b})) = 0 then 1 else 0 end as asset_b_is_null
    
    , case
            when
                {a} is not null 
                and {b} is not null
                and {a} = {b} then 1
            when {a} is null and {b} is null then -1
            else 0
    end as asset_eq
    
    , if (message_id is null, 1, 0) as miss_madmen
    , IS_AUDIT_SKIPPED
    
    , count(0) as num

from pypl-bods.rsh_row_gds_focus.acdc_component_simu_driver_20221201_20221231_variables 

group by 1,2,3,4,5,6,7,8
order by 1,2,3,4,5,6,7,8

## sql simulate gainchart

In [ ]:
sql_score_op = """
select
    '{score}' as score_name
    , case
        when {score} >= 700 then cast(floor({score} / 5) as int) * 5
        else 0
    end as score_bucket
    
    , sum(if(is_cc_bad in (1,2), capture_usd_amt, 0))          as bad_amt
    , sum(if(is_cc_bad in (1,2), 1, 0))                        as bad_cnt
    
    , sum(if(is_cc_bad in (0), capture_usd_amt, 0))            as good_amt
    , sum(if(is_cc_bad in (0), 1, 0))                          as good_cnt
    , sum(capture_usd_amt)                                     as amt
    , count(0)                                                 as cnt
    
from pypl-bods.rsh_row_gds_focus.ucc_decline_pattr_txn_model_score
where
    is_cg = 1
    and decline_type in ('Approved')
    
    
group by 1,2
order by 1,2
"""

scores = [
    'MDL7_UCC21_model_score1'
    'MDL7_UCC_SF_STATIC_RMR_score'
    'MDL5_CAM22_EMS_SCORE'
]

score_op_df = pd.DataFrame()
for score, _ in scores:
    print(score)
    sql_cmd = sql_score_op.format(score=score)
    print(sql_cmd)
    df = %ppbq $sql_cmd
    score_op_df = pd.concat([score_op_df, df])



### cut score bin

In [ ]:
sql_score_bin = """
case
    when ucc23_v2_model_score1 >= 700 then format("%.*f", 0, cast(floor(ucc23_v2_model_score1 / 10) as int) * 10.0)
    else format("%.*f", 0, cast(floor(ucc23_v2_model_score1 / 100) as int) * 100.0)
end
"""

## debug table

if order by column has null value:
- order by asc, null value come first
- order by des, null value come last


In [ ]:
WITH finishers AS
 (
SELECT
                'Sophia Liu' as name, TIMESTAMP '2016-10-18 2:51:45' as finish_time, 'F30-34' as division
UNION ALL SELECT 'Lisa Stelzner', TIMESTAMP '2016-10-18 2:54:11', 'F35-39'
UNION ALL SELECT 'Nikki Leith', TIMESTAMP '2016-10-18 2:59:01', 'F30-34'
UNION ALL SELECT 'Lauren Matthews', TIMESTAMP '2016-10-18 3:01:17', 'F35-39'
UNION ALL SELECT 'Desiree Berry', TIMESTAMP '2016-10-18 3:05:42', 'F35-39'
UNION ALL SELECT 'Suzy Slane', TIMESTAMP '2016-10-18 3:06:24', 'F35-39'
UNION ALL SELECT 'Jen Edwards', TIMESTAMP '2016-10-18 3:06:36', 'F30-34'
UNION ALL SELECT 'Meghan Lederer', NULL, 'F30-34'
)
SELECT
    name,
    finish_time,

    division,
    ROW_NUMBER() OVER (PARTITION BY division ORDER BY finish_time DESC) AS finish_rank
FROM finishers;



## set table partition expiration date

In [ ]:
ALTER TABLE pypl-bods.rsh_row_gds_focus.table_name SET OPTIONS (partition_expiration_days = 360);


## id duplication test

In [ ]:
%%ppbq
-- id duplication test

select 
    group_size
    , count(0) as num
from (
select
    public_id
    , count(0) as group_size
from pypl-bods.pp_risk_rap_bt_managed_pii_views.bt_txn_uds
group by 1
) a

group by 1
order by 2 desc

limit 100


## record dedup

In [ ]:
select
    *
from table_name
QUALIFY ROW_NUMBER() OVER (PARTITION BY trans_id ORDER BY pmt_start_ts) = 1

## generate array and flatten into rows

In [ ]:
SELECT *
FROM UNNEST(GENERATE_ARRAY(0, 10, 1)) AS element
WITH OFFSET AS offset
ORDER BY offset

## truncate date to month first day

In [ ]:
DATE_TRUNC(date_column, month) as monthly

# create a dt partitioned table from parquet file

In [ ]:
CREATE EXTERNAL TABLE pypl-edw.pp_fastr_gds_tables.ten_v2_pred
WITH PARTITION COLUMNS(
    dt string
)
OPTIONS(
format='PARQUET',
decimal_target_types=['BIGNUMERIC'],
hive_partition_uri_prefix='gs://pypl-bkt-prd-row-std-zonkey-pride/apps/risk/ars/zonkeybooks_db/ten_v2_pred',
uris=['gs://pypl-bkt-prd-row-std-zonkey-pride/apps/risk/ars/zonkeybooks_db/ten_v2_pred/*']
);


# build table from parquet files

In [ ]:
drop table if exists pypl-bods.rsh_row_gds_focus.guxia_honey_users;

create or replace external table pypl-bods.rsh_row_gds_focus.guxia_honey_users 
options (
    format="PARQUET",
    enum_as_string=true,
    decimal_target_types=["NUMERIC", "BIGNUMERIC"],
    uris = ["gs://pypl-bkt-rsh-row-std-gds-focus/path/to/copied/data/dir/*.parquet"]
)